<a href="https://colab.research.google.com/github/srirambtechit/msc-project/blob/main/Project_Cicket_HighLight_Video_Generation_Model_Detectron2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cricket HighLight Video Generation by detecting Umpire gestures and poses

### Steps followed

In notebook, we will walk through the steps required to train Detectron2 on  custom umpire dataset.

To train our detector we take the following steps:

* Install Detectron2 dependencies
* Current colab environment details
* Download custom Detectron2 object detection data
* Visualize Detectron2 training data
* Write our Detectron2 Training configuration
* Run Detectron2 training
* Evaluate Detectron2 performance
* Run Detectron2 inference on test images


# Install Detectron2 Dependencies

In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 25kB/s 
     |████████████████████████████████| 6.6MB 18.8MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 276kB 8.7MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=8f38481274d3af33182336bc1c998b669a06e72223be41c4d9fdd4b72c670270
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip

In [2]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2MB 3.9MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201122-cp36-none-any.whl size=44456 sha256=0fe271aca087bcb847733cee68cf0d07313807c12fd6f841c4a183974f554075
  Stored in directory: /root/.cache/pip/wheels/33/7b/ec/59dffd0a0aa995739aad60ac97e70252247e9d17eae69c1cee
Successfully built fvcore


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

# Current colab environment details

In [3]:
# Detectron2 environment details
!python3 -m detectron2.utils.collect_env

------------------------  ---------------------------------------------------------------
sys.platform              linux
Python                    3.6.9 (default, Oct  8 2020, 12:12:24) [GCC 8.4.0]
numpy                     1.18.5
detectron2                0.1.3 @/usr/local/lib/python3.6/dist-packages/detectron2
detectron2 compiler       GCC 7.3
detectron2 CUDA compiler  10.1
detectron2 arch flags     sm_35, sm_37, sm_50, sm_52, sm_60, sm_61, sm_70, sm_75
DETECTRON2_ENV_MODULE     <not set>
PyTorch                   1.5.0+cu101 @/usr/local/lib/python3.6/dist-packages/torch
PyTorch debug build       False
CUDA available            True
GPU 0                     Tesla P100-PCIE-16GB
CUDA_HOME                 /usr/local/cuda
NVCC                      Cuda compilation tools, release 10.1, V10.1.243
Pillow                    7.0.0
torchvision               0.6.0+cu101 @/usr/local/lib/python3.6/dist-packages/torchvision
torchvision arch flags    sm_35, sm_50, sm_60, sm_70, sm_75
fvcore     

In [4]:
# Current allocated GPU configuration
!nvidia-smi

Tue Dec  1 12:15:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download custom Detectron2 object detection data

In [12]:
!rm -rf sample_data
!git clone https://github.com/srirambtechit/msc-project.git; mv msc-project/* /content; rm -rf msc-project

Cloning into 'msc-project'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 3046 (delta 2), reused 13 (delta 2), pack-reused 3029
Receiving objects: 100% (3046/3046), 249.67 MiB | 48.99 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Downloading trained models from drive
!cp -r /content/drive/MyDrive/models/ .

In [13]:
# Install dependencies, download the video, and crop 5 seconds for processing
!pip install youtube-dl
!pip uninstall -y opencv-python-headless opencv-contrib-python
!apt install python3-opencv  # the one pre-installed have some issues

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-opencv is already the newest version (3.2.0+dfsg-4ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
!youtube-dl https://youtu.be/713GON0Ru7k -f 22 -o video.mp4
!ffmpeg -loglevel quiet -i video.mp4 -t 00:00:08 -c:v copy video-clip.mp4

In [117]:
# delete if exists
![[ -f video-output-u.mp4 ]] && rm -f video-output-u.mp4

# model1 - umpire detection
!python detectron2/demo.py \
  --config-file detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml \
  --video-input video-clip.mp4 \
  --output video-output-u.mp4 \
  --classifier umpire-classifier \
  --confidence-threshold 0.6 \
  --opts MODEL.WEIGHTS /content/models/model1/output/model_final.pth MODEL.ROI_HEADS.NUM_CLASSES 3

Failed to load OpenCL runtime
[12/01 17:48:02 detectron2]: Arguments: Namespace(classifier='umpire-classifier', confidence_threshold=0.6, config_file='detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', input=None, opts=['MODEL.WEIGHTS', '/content/models/model1/output/model_final.pth', 'MODEL.ROI_HEADS.NUM_CLASSES', '3'], output='video-output-u.mp4', video_input='video-clip.mp4', webcam=False)
[12/01 17:48:07 fvcore.common.checkpoint]: Loading checkpoint from /content/models/model1/output/model_final.pth
OpenCV: FFMPEG: tag 0x34363278/'x264' is not supported with codec id 28 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
  0% 0/241 [00:00<?, ?it/s]Traceback (most recent call last):
  File "detectron2/demo.py", line 130, in <module>
    for vis_frame in tqdm.tqdm(demo.run_on_video(video), total=num_frames):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1104, in __iter__
    for obj in iterable:
  File

In [40]:
# delete if exists
![[ -f video-output-up.mp4 ]] && rm -f video-output-up.mp4

# model2 - umpire pose detection
!python detectron2/demo.py \
  --config-file detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml \
  --video-input video-clip.mp4 \
  --output video-output-up.mp4 \
  --classifier umpire-pose-classifier \
  --confidence-threshold 0.6 \
  --opts MODEL.WEIGHTS /content/models/model2/output/model_final.pth MODEL.ROI_HEADS.NUM_CLASSES 7

Failed to load OpenCL runtime
[12/01 13:53:37 detectron2]: Arguments: Namespace(classifier='umpire-pose-classifier', confidence_threshold=0.6, config_file='detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', input=None, opts=['MODEL.WEIGHTS', '/content/models/model2/output/model_final.pth', 'MODEL.ROI_HEADS.NUM_CLASSES', '7'], output='video-output-up.mp4', video_input='video-clip.mp4', webcam=False)
Metadata(name='my_dataset_val', thing_classes=['umpire-signals', 'no-action', 'no-ball', 'out', 'six', 'wide'], thing_dataset_id_to_contiguous_id={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5})
[12/01 13:53:42 fvcore.common.checkpoint]: Loading checkpoint from /content/models/model2/output/model_final.pth
OpenCV: FFMPEG: tag 0x34363278/'x264' is not supported with codec id 28 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
100% 241/241 [01:12<00:00,  3.33it/s]


In [123]:
# delete if exists
![[ -f video-output.mp4 ]] && rm -f video-output.mp4

# model1 - umpire detection
!python detectron2/demo.py \
  --config-file detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml \
  --video-input video-clip.mp4 \
  --output video-output.mp4 \
  --confidence-threshold 0.6 

Failed to load OpenCL runtime
[12/01 17:53:13 detectron2]: Arguments: Namespace(classifier='both', confidence_threshold=0.6, config_file='detectron2/configs/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', input=None, opts=[], output='video-output.mp4', video_input='video-clip.mp4', webcam=False)
[12/01 17:53:18 fvcore.common.checkpoint]: Loading checkpoint from /content/models/model1/output/model_final.pth
[12/01 17:53:20 fvcore.common.checkpoint]: Loading checkpoint from /content/models/model2/output/model_final.pth
OpenCV: FFMPEG: tag 0x34363278/'x264' is not supported with codec id 28 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
  8% 19/241 [00:04<00:50,  4.38it/s]Traceback (most recent call last):
  File "detectron2/demo.py", line 130, in <module>
    for vis_frame in tqdm.tqdm(demo.run_on_video(video), total=num_frames):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1104, in __iter__
    for obj in iterable:

In [108]:
from IPython.display import HTML
from base64 import b64encode
# mp4 = open('video-clip.mp4','rb').read()
mp4 = open('video-output.mp4','rb').read()
# mp4 = open('video-output-u.mp4','rb').read()
# mp4 = open('video-output-up.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Developement space

In [1]:
# !git clone https://github.com/srirambtechit/msc-project.git; 
!cd msc-project; git pull origin; cp -r detectron2 /content

/bin/bash: line 0: cd: msc-project: No such file or directory
fatal: not a git repository (or any of the parent directories): .git
cp: cannot stat 'detectron2': No such file or directory


In [110]:
data_list = [1, 3]
result = torch.tensor(data_list)
print(result)
print(result.shape == torch.Size([2]))
print(torch.tensor([2]) in result)

print('----')

print(2 in torch.tensor([1,3, 2]))
print(2 not in torch.tensor([1,3]))
print(2 in torch.tensor([2]))
print(2 in torch.tensor([2,2]))
print(2 not in torch.tensor([]))

tensor([1, 3])
True
False
----
True
True
True
True
True
